.**Project 2 : Sentiment analysis on 1.6 million tweets**

In [43]:
pip install transformers torch

In [24]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset
import torch

In [16]:
import pandas as pd
df = pd.read_csv('/training.1600000.processed.noemoticon.csv', encoding='latin-1', header=None, names=['target', 'ids', 'date', 'flag', 'user', 'text'])
df = df[['target', 'text']]

In [17]:
df = df[['target', 'text']]

In [18]:
df['text'] = df['text'].str.lower().replace('[^a-zA-Z0-9\s]', '', regex=True)

In [19]:
# Split the data into train and validation sets
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

In [20]:
# Tokenize and encode the text data
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [25]:
# Tokenize and encode the training set
train_texts = list(train_df['text'].astype(str))  # Convert to strings
train_labels = list(train_df['target'].astype(int))  # Convert to integers

train_tokens = tokenizer(train_texts, padding=True, truncation=True, return_tensors='pt')
train_dataset = TensorDataset(train_tokens['input_ids'], train_tokens['attention_mask'], torch.tensor(train_labels))


In [26]:
# Tokenize and encode the validation set
val_texts = list(val_df['text'].astype(str))  # Convert to strings
val_labels = list(val_df['target'])

val_tokens = tokenizer(val_texts, padding=True, truncation=True, return_tensors='pt')
val_dataset = TensorDataset(val_tokens['input_ids'], val_tokens['attention_mask'], torch.tensor(val_labels))


In [27]:
# Prepare DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [28]:
# Load pre-trained BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)  # Assuming binary classification (negative and positive)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
# Define optimizer and loss function
optimizer = AdamW(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [30]:
# Determine device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [31]:
# Move the model to the selected device
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [33]:
from tqdm import tqdm
epochs = 1
for epoch in range(epochs):
    model.train()

    # Iterate over batches
    for batch in tqdm(train_dataloader, desc=f'Epoch {epoch + 1}/{epochs}'):
        # Move batch data to the selected device
        input_ids, attention_mask, targets = batch
        input_ids, attention_mask, targets = input_ids.to(device), attention_mask.to(device), targets.to(device)

        # Zero gradients, forward pass, backward pass, optimizer step
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=targets)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

Epoch 1/1: 100%|██████████| 8770/8770 [56:13<00:00,  2.60it/s]


In [34]:
# Evaluate the model on the validation set
model.eval()
with torch.no_grad():
    val_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    for batch in tqdm(val_dataloader, desc='Validation'):
        input_ids, attention_mask, targets = batch
        input_ids, attention_mask, targets = input_ids.to(device), attention_mask.to(device), targets.to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=targets)
        loss = outputs.loss
        val_loss += loss.item()

        predictions = torch.argmax(outputs.logits, dim=1)
        correct_predictions += (predictions == targets).sum().item()
        total_samples += targets.size(0)

    average_val_loss = val_loss / len(val_dataloader)
    accuracy = correct_predictions / total_samples
    print(f'Validation Loss: {average_val_loss:.4f}, Accuracy: {accuracy:.4f}')

Validation: 100%|██████████| 2193/2193 [04:19<00:00,  8.44it/s]

Validation Loss: 0.0000, Accuracy: 1.0000


In [35]:
print(f'Validation Loss: {average_val_loss:.4f}, Accuracy: {accuracy:.4f}')

Validation Loss: 0.0000, Accuracy: 1.0000


In [38]:
# Sample tweets for testing
sample_tweets = [ 'Not satisfied with the service.']

# Tokenize and encode the sample tweets
sample_tokens = tokenizer(sample_tweets, padding=True, truncation=True, return_tensors='pt')

# Move the data to the selected device (CPU or GPU)
sample_tokens = {key: value.to(device) for key, value in sample_tokens.items()}

# Make predictions on the sample tweets
model.eval()
with torch.no_grad():
    sample_outputs = model(**sample_tokens)
    sample_predictions = torch.argmax(sample_outputs.logits, dim=1)

# Convert predictions to a list
sample_predicted_labels = sample_predictions.tolist()

# Display the results
for tweet, label in zip(sample_tweets, sample_predicted_labels):
    print(f'Tweet: {tweet}, Predicted Label: {label}')


Tweet: Not satisfied with the service., Predicted Label: 0
